In [61]:
# standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
import time

# create lstm models
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error

# hyperparameter tuning
from kerastuner.tuners import RandomSearch
from tensorflow.keras.callbacks import EarlyStopping, Callback

# 1-Load Data

In [13]:
# load interim data set
data_set = pd.read_csv('../data/interim/international-airline-passengers.csv')
univariate_data_set = data_set["passenger_number"]
univariate_data_set

0      112.0
1      118.0
2      132.0
3      129.0
4      121.0
       ...  
139    606.0
140    508.0
141    461.0
142    390.0
143    432.0
Name: passenger_number, Length: 144, dtype: float64

In [16]:
univariate_array = univariate_data_set.values
univariate_array

array([112., 118., 132., 129., 121., 135., 148., 148., 136., 119., 104.,
       118., 115., 126., 141., 135., 125., 149., 170., 170., 158., 133.,
       114., 140., 145., 150., 178., 163., 172., 178., 199., 199., 184.,
       162., 146., 166., 171., 180., 193., 181., 183., 218., 230., 242.,
       209., 191., 172., 194., 196., 196., 236., 235., 229., 243., 264.,
       272., 237., 211., 180., 201., 204., 188., 235., 227., 234., 264.,
       302., 293., 259., 229., 203., 229., 242., 233., 267., 269., 270.,
       315., 364., 347., 312., 274., 237., 278., 284., 277., 317., 313.,
       318., 374., 413., 405., 355., 306., 271., 306., 315., 301., 356.,
       348., 355., 422., 465., 467., 404., 347., 305., 336., 340., 318.,
       362., 348., 363., 435., 491., 505., 404., 359., 310., 337., 360.,
       342., 406., 396., 420., 472., 548., 559., 463., 407., 362., 405.,
       417., 391., 419., 461., 472., 535., 622., 606., 508., 461., 390.,
       432.])

# 2-Normalize Data

In [19]:
univariate_array_reshaped = univariate_array.reshape(-1, 1)
univariate_array_reshaped

array([[112.],
       [118.],
       [132.],
       [129.],
       [121.],
       [135.],
       [148.],
       [148.],
       [136.],
       [119.],
       [104.],
       [118.],
       [115.],
       [126.],
       [141.],
       [135.],
       [125.],
       [149.],
       [170.],
       [170.],
       [158.],
       [133.],
       [114.],
       [140.],
       [145.],
       [150.],
       [178.],
       [163.],
       [172.],
       [178.],
       [199.],
       [199.],
       [184.],
       [162.],
       [146.],
       [166.],
       [171.],
       [180.],
       [193.],
       [181.],
       [183.],
       [218.],
       [230.],
       [242.],
       [209.],
       [191.],
       [172.],
       [194.],
       [196.],
       [196.],
       [236.],
       [235.],
       [229.],
       [243.],
       [264.],
       [272.],
       [237.],
       [211.],
       [180.],
       [201.],
       [204.],
       [188.],
       [235.],
       [227.],
       [234.],
       [264.],
       [30

In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_univariate_array = scaler.fit_transform(univariate_array_reshaped)

# 3-Perform Train (70%), Validation (18%), Test Split (12%)

In [51]:
train_size = int(len(normalized_univariate_array) * 0.7)
validation_size = int(len(normalized_univariate_array) * 0.18)
test_size = len(normalized_univariate_array) - train_size - validation_size
train_set, validation_set, test_set = normalized_univariate_array[0:train_size,:], normalized_univariate_array[train_size:train_size+validation_size,:], normalized_univariate_array[train_size+validation_size:len(normalized_univariate_array),:]
print(f"Training set: {len(train_set)}, validation set: {len(validation_set)}, test set: {len(test_set)}")

Training set: 100, validation set: 25, test set: 19


# 4-Create samples using Moving Window Technique

In [87]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix][0]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

## Let's set size of the input window width to 12, becuase of the strong yearly seasonality

In [88]:
input_window_size = 12

train_samples = split_sequence(train_set, input_window_size, 1)

val_samples = split_sequence(validation_set, input_window_size, 1)
    
test_samples = split_sequence(test_set, input_window_size, 1)

print(f"Number of training samples: {len(train_samples[0])}, validation samples: {len(val_samples[0])}, test samples: {len(test_samples[0])}")

Number of training samples: 88, validation samples: 13, test samples: 7


In [89]:
train_samples

(array([[[0.01544402],
         [0.02702703],
         [0.05405405],
         ...,
         [0.02895753],
         [0.        ],
         [0.02702703]],
 
        [[0.02702703],
         [0.05405405],
         [0.04826255],
         ...,
         [0.        ],
         [0.02702703],
         [0.02123552]],
 
        [[0.05405405],
         [0.04826255],
         [0.03281853],
         ...,
         [0.02702703],
         [0.02123552],
         [0.04247104]],
 
        ...,
 
        [[0.33397683],
         [0.41119691],
         [0.4034749 ],
         ...,
         [0.32239382],
         [0.38996139],
         [0.40733591]],
 
        [[0.41119691],
         [0.4034749 ],
         [0.41312741],
         ...,
         [0.38996139],
         [0.40733591],
         [0.38030888]],
 
        [[0.4034749 ],
         [0.41312741],
         [0.52123552],
         ...,
         [0.40733591],
         [0.38030888],
         [0.48648649]]]),
 array([[0.02123552],
        [0.04247104],
        [0.

# 5-Network Architecture

In [90]:
def build_model(hp):
    model = Sequential()
    # input LSTM layer
    model.add(LSTM(hp.Int("input_units", min_value=5, max_value=50, step=3),
                   activation='relu', input_shape=(input_window_size, 1), name="lstm"))
    
    # dense hidden layers with dropouts
    for i in range(hp.Int("n_layers", 0,2)):
        model.add(Dense(hp.Int(f"dense_{i}_units", min_value=5, max_value=50, step=3),
                        activation='relu', name=f"hidden_{i}"))
        for j in range(hp.Int("dropout_layers", 0, 1)):
            model.add(Dropout(hp.Choice(f"drop_rate_{j}", values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]), name=f"dropout_{i}{j}"))
    
    # dense output layer
    model.add(Dense(1, name="output"))
    model.compile(loss='mean_squared_error' , optimizer=Adam(learning_rate=
            hp.Choice("learning_rate",values=[1e-2, 1e-3, 1e-4])), metrics=[metrics.mae])
    return model

In [91]:
approach = "univariate"

# training & hyperparameter tuning
LOG_DIR = f"{int((time.time()))}"

tuner = RandomSearch(
        build_model,
        objective="val_mean_absolute_error", # evtl. RMSE
        max_trials= 5,
        executions_per_trial=3,
        directory='one_step_univariate/tuning_logs',
        project_name=LOG_DIR
    )
    
tuner.search(
    x=train_samples[0],
    y=train_samples[1],
    epochs=100,
    batch_size=32,
    validation_data=(val_samples[0], val_samples[1]),
    callbacks=[EarlyStopping('val_loss', patience=5)])
    
# get best model
model = tuner.get_best_models(num_models=1)[0]
model.save(f"../models/{approach}_{input_window_size}")

Train on 88 samples, validate on 13 samples
Epoch 1/100
88/88 [==============================] - ETA: 1s - loss: 0.1056 - mean_absolute_error: 0.281 - 1s 10ms/sample - loss: 0.0954 - mean_absolute_error: 0.2702 - val_loss: 0.3698 - val_mean_absolute_error: 0.5980
Epoch 2/100
88/88 [==============================] - ETA: 0s - loss: 0.0799 - mean_absolute_error: 0.246 - 0s 328us/sample - loss: 0.0816 - mean_absolute_error: 0.2465 - val_loss: 0.3276 - val_mean_absolute_error: 0.5616
Epoch 3/100
88/88 [==============================] - ETA: 0s - loss: 0.0656 - mean_absolute_error: 0.221 - 0s 354us/sample - loss: 0.0697 - mean_absolute_error: 0.2244 - val_loss: 0.2905 - val_mean_absolute_error: 0.5275
Epoch 4/100
88/88 [==============================] - ETA: 0s - loss: 0.0835 - mean_absolute_error: 0.245 - 0s 320us/sample - loss: 0.0595 - mean_absolute_error: 0.2026 - val_loss: 0.2550 - val_mean_absolute_error: 0.4926
Epoch 5/100
88/88 [==============================] - ETA: 0s - loss: 0.06

Epoch 14/100
88/88 [==============================] - ETA: 0s - loss: 0.0054 - mean_absolute_error: 0.058 - 0s 257us/sample - loss: 0.0057 - mean_absolute_error: 0.0626 - val_loss: 0.0197 - val_mean_absolute_error: 0.1080
Epoch 15/100
88/88 [==============================] - ETA: 0s - loss: 0.0070 - mean_absolute_error: 0.070 - 0s 241us/sample - loss: 0.0055 - mean_absolute_error: 0.0592 - val_loss: 0.0221 - val_mean_absolute_error: 0.1135
Epoch 16/100
88/88 [==============================] - ETA: 0s - loss: 0.0066 - mean_absolute_error: 0.067 - 0s 253us/sample - loss: 0.0054 - mean_absolute_error: 0.0573 - val_loss: 0.0236 - val_mean_absolute_error: 0.1166
Train on 88 samples, validate on 13 samples
Epoch 1/100
88/88 [==============================] - ETA: 1s - loss: 0.0872 - mean_absolute_error: 0.267 - 1s 10ms/sample - loss: 0.0744 - mean_absolute_error: 0.2380 - val_loss: 0.2914 - val_mean_absolute_error: 0.5287
Epoch 2/100
88/88 [==============================] - ETA: 0s - loss: 0

Train on 88 samples, validate on 13 samples
Epoch 1/100
88/88 [==============================] - ETA: 1s - loss: 0.0709 - mean_absolute_error: 0.242 - 1s 10ms/sample - loss: 0.0739 - mean_absolute_error: 0.2380 - val_loss: 0.3049 - val_mean_absolute_error: 0.5417
Epoch 2/100
88/88 [==============================] - ETA: 0s - loss: 0.0756 - mean_absolute_error: 0.240 - 0s 346us/sample - loss: 0.0728 - mean_absolute_error: 0.2359 - val_loss: 0.3013 - val_mean_absolute_error: 0.5384
Epoch 3/100
88/88 [==============================] - ETA: 0s - loss: 0.0846 - mean_absolute_error: 0.252 - 0s 370us/sample - loss: 0.0717 - mean_absolute_error: 0.2338 - val_loss: 0.2978 - val_mean_absolute_error: 0.5351
Epoch 4/100
88/88 [==============================] - ETA: 0s - loss: 0.0801 - mean_absolute_error: 0.247 - 0s 326us/sample - loss: 0.0706 - mean_absolute_error: 0.2317 - val_loss: 0.2944 - val_mean_absolute_error: 0.5318
Epoch 5/100
88/88 [==============================] - ETA: 0s - loss: 0.07

Epoch 38/100
88/88 [==============================] - ETA: 0s - loss: 0.0372 - mean_absolute_error: 0.162 - 0s 392us/sample - loss: 0.0417 - mean_absolute_error: 0.1684 - val_loss: 0.1955 - val_mean_absolute_error: 0.4286
Epoch 39/100
88/88 [==============================] - ETA: 0s - loss: 0.0445 - mean_absolute_error: 0.188 - 0s 353us/sample - loss: 0.0409 - mean_absolute_error: 0.1666 - val_loss: 0.1929 - val_mean_absolute_error: 0.4256
Epoch 40/100
88/88 [==============================] - ETA: 0s - loss: 0.0452 - mean_absolute_error: 0.175 - 0s 331us/sample - loss: 0.0402 - mean_absolute_error: 0.1646 - val_loss: 0.1903 - val_mean_absolute_error: 0.4225
Epoch 41/100
88/88 [==============================] - ETA: 0s - loss: 0.0290 - mean_absolute_error: 0.136 - 0s 355us/sample - loss: 0.0394 - mean_absolute_error: 0.1626 - val_loss: 0.1877 - val_mean_absolute_error: 0.4194
Epoch 42/100
88/88 [==============================] - ETA: 0s - loss: 0.0606 - mean_absolute_error: 0.211 - 0s 3

Epoch 75/100
88/88 [==============================] - ETA: 0s - loss: 0.0114 - mean_absolute_error: 0.078 - 0s 411us/sample - loss: 0.0115 - mean_absolute_error: 0.0815 - val_loss: 0.0720 - val_mean_absolute_error: 0.2457
Epoch 76/100
88/88 [==============================] - ETA: 0s - loss: 0.0071 - mean_absolute_error: 0.068 - 0s 360us/sample - loss: 0.0109 - mean_absolute_error: 0.0799 - val_loss: 0.0686 - val_mean_absolute_error: 0.2387
Epoch 77/100
88/88 [==============================] - ETA: 0s - loss: 0.0116 - mean_absolute_error: 0.082 - 0s 344us/sample - loss: 0.0105 - mean_absolute_error: 0.0785 - val_loss: 0.0653 - val_mean_absolute_error: 0.2315
Epoch 78/100
88/88 [==============================] - ETA: 0s - loss: 0.0128 - mean_absolute_error: 0.091 - 0s 364us/sample - loss: 0.0101 - mean_absolute_error: 0.0771 - val_loss: 0.0621 - val_mean_absolute_error: 0.2245
Epoch 79/100
88/88 [==============================] - ETA: 0s - loss: 0.0119 - mean_absolute_error: 0.089 - 0s 4

...

Epoch 97/100
88/88 [==============================] - ETA: 0s - loss: 0.0086 - mean_absolute_error: 0.073 - 0s 257us/sample - loss: 0.0093 - mean_absolute_error: 0.0742 - val_loss: 0.0613 - val_mean_absolute_error: 0.2209
Epoch 98/100
88/88 [==============================] - ETA: 0s - loss: 0.0115 - mean_absolute_error: 0.079 - 0s 257us/sample - loss: 0.0098 - mean_absolute_error: 0.0789 - val_loss: 0.0594 - val_mean_absolute_error: 0.2166
Epoch 99/100
88/88 [==============================] - ETA: 0s - loss: 0.0081 - mean_absolute_error: 0.065 - 0s 248us/sample - loss: 0.0094 - mean_absolute_error: 0.0730 - val_loss: 0.0576 - val_mean_absolute_error: 0.2124
Epoch 100/100
88/88 [==============================] - ETA: 0s - loss: 0.0078 - mean_absolute_error: 0.072 - 0s 289us/sample - loss: 0.0086 - mean_absolute_error: 0.0716 - val_loss: 0.0560 - val_mean_absolute_error: 0.2087


Train on 88 samples, validate on 13 samples
Epoch 1/100
88/88 [==============================] - ETA: 1s - loss: 0.0887 - mean_absolute_error: 0.257 - 1s 10ms/sample - loss: 0.0882 - mean_absolute_error: 0.2598 - val_loss: 0.3430 - val_mean_absolute_error: 0.5758
Epoch 2/100
88/88 [==============================] - ETA: 0s - loss: 0.0933 - mean_absolute_error: 0.266 - 0s 361us/sample - loss: 0.0757 - mean_absolute_error: 0.2379 - val_loss: 0.3061 - val_mean_absolute_error: 0.5427
Epoch 3/100
88/88 [==============================] - ETA: 0s - loss: 0.0934 - mean_absolute_error: 0.270 - 0s 359us/sample - loss: 0.0678 - mean_absolute_error: 0.2230 - val_loss: 0.2847 - val_mean_absolute_error: 0.5225
Epoch 4/100
88/88 [==============================] - ETA: 0s - loss: 0.0590 - mean_absolute_error: 0.213 - 0s 310us/sample - loss: 0.0612 - mean_absolute_error: 0.2097 - val_loss: 0.2635 - val_mean_absolute_error: 0.5018
Epoch 5/100
88/88 [==============================] - ETA: 0s - loss: 0.04

Epoch 38/100
88/88 [==============================] - ETA: 0s - loss: 0.0033 - mean_absolute_error: 0.044 - 0s 291us/sample - loss: 0.0038 - mean_absolute_error: 0.0477 - val_loss: 0.0147 - val_mean_absolute_error: 0.1000
Epoch 39/100
88/88 [==============================] - ETA: 0s - loss: 0.0040 - mean_absolute_error: 0.048 - 0s 283us/sample - loss: 0.0039 - mean_absolute_error: 0.0482 - val_loss: 0.0148 - val_mean_absolute_error: 0.1007
Epoch 40/100
88/88 [==============================] - ETA: 0s - loss: 0.0046 - mean_absolute_error: 0.052 - 0s 270us/sample - loss: 0.0038 - mean_absolute_error: 0.0477 - val_loss: 0.0141 - val_mean_absolute_error: 0.0970
Epoch 41/100
88/88 [==============================] - ETA: 0s - loss: 0.0047 - mean_absolute_error: 0.050 - 0s 281us/sample - loss: 0.0038 - mean_absolute_error: 0.0474 - val_loss: 0.0139 - val_mean_absolute_error: 0.0955
Epoch 42/100
88/88 [==============================] - ETA: 0s - loss: 0.0034 - mean_absolute_error: 0.043 - 0s 2

Epoch 11/100
88/88 [==============================] - ETA: 0s - loss: 0.0128 - mean_absolute_error: 0.101 - 0s 255us/sample - loss: 0.0105 - mean_absolute_error: 0.0880 - val_loss: 0.0178 - val_mean_absolute_error: 0.1005
Epoch 12/100
88/88 [==============================] - ETA: 0s - loss: 0.0091 - mean_absolute_error: 0.081 - 0s 234us/sample - loss: 0.0101 - mean_absolute_error: 0.0861 - val_loss: 0.0214 - val_mean_absolute_error: 0.1106
Epoch 13/100
88/88 [==============================] - ETA: 0s - loss: 0.0090 - mean_absolute_error: 0.082 - 0s 265us/sample - loss: 0.0083 - mean_absolute_error: 0.0761 - val_loss: 0.0281 - val_mean_absolute_error: 0.1315
Epoch 14/100
88/88 [==============================] - ETA: 0s - loss: 0.0061 - mean_absolute_error: 0.064 - 0s 250us/sample - loss: 0.0074 - mean_absolute_error: 0.0684 - val_loss: 0.0350 - val_mean_absolute_error: 0.1528
Epoch 15/100
88/88 [==============================] - ETA: 0s - loss: 0.0070 - mean_absolute_error: 0.067 - 0s 2

INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Assets written to: ../models/univariate_12/assets


# 6-Make &Denormalize Predictions

In [92]:
# make predictions
predictions_for_train = model.predict(train_samples[0])
predictions_for_validation = model.predict(val_samples[0])
predictions_for_test = model.predict(test_samples[0])
predictions_for_test

array([[0.7153004 ],
       [0.7388431 ],
       [0.76644963],
       [0.78308517],
       [0.79344517],
       [0.8021602 ],
       [0.7879256 ]], dtype=float32)

In [93]:
# invert predictions
predictions_for_train_denormalized = scaler.inverse_transform(predictions_for_train)
true_values_train = scaler.inverse_transform(train_samples[1])

predictions_for_val_denormalized = scaler.inverse_transform(predictions_for_validation)
true_values_val = scaler.inverse_transform(val_samples[1])

predictions_for_test_denormalized = scaler.inverse_transform(predictions_for_test)
true_values_test = scaler.inverse_transform(test_samples[1])

# 7-Evaluation

In [104]:
# calculate root mean squared error
train_score = math.sqrt(mean_squared_error(true_values_train, predictions_for_train_denormalized))
print('Train Score: %.2f RMSE' % (train_score))

validation_score = math.sqrt(mean_squared_error(true_values_val, predictions_for_val_denormalized))
print('Validation Score: %.2f RMSE' % (validation_score))

test_score = math.sqrt(mean_squared_error(true_values_test, predictions_for_test_denormalized))
print('Test Score: %.2f RMSE' % (test_score))

Train Score: 31.19 RMSE
Validation Score: 58.81 RMSE
Test Score: 91.88 RMSE


In [107]:
# shift train predictions for plotting
train_predict_plot = np.empty_like(normalized_univariate_array)
train_predict_plot[:, :] = np.nan
train_predict_plot[input_window_size:len(predictions_for_train_denormalized)+input_window_size, :] = predictions_for_train_denormalized

# shift validation predictions for plotting
validation_predict_plot = np.empty_like(normalized_univariate_array)
validation_predict_plot[:, :] = np.nan
validation_predict_plot[len(predictions_for_train_denormalized)+(input_window_size*2)+1:len(normalized_univariate_array)-1, :] = predictions_for_val_denormalized

# plot baseline and predictions
plt.plot(scaler.inverse_transform(normalized_univariate_array))
plt.plot(train_predict_plot)
plt.plot(validation_predict_plot)
plt.show()

ValueError: could not broadcast input array from shape (13,1) into shape (30,1)